In [ ]:
import pandas as pd
import sagemaker
 
print (sagemaker.__version__)

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()                     
prefix = 'sagemaker/DEMO-autopilot-insurance-claims'

In [ ]:
claims_data = pd.read_csv('claims.csv')

In [ ]:
claims_data

In [ ]:
customers_data = pd.read_csv('customers.csv')

In [ ]:
customers_data

In [ ]:
dataset = claims_data.join(customers_data, how='left', on='policy_id', lsuffix='_other')

In [ ]:
dataset.drop(['policy_id_other', 'policy_id'], axis=1, inplace=True)

In [ ]:
dataset

In [ ]:
dataset.to_csv('dataset_autopilot.csv', index=False, header=True)

In [ ]:
dataset_path = sess.upload_data(path='dataset_autopilot.csv', key_prefix=prefix + '/input/training')
print(dataset_path)

In [ ]:
from sagemaker.automl.automl import AutoML

automl_job = AutoML(
    role=role,
    target_attribute_name='fraud',
    problem_type='BinaryClassification',
    job_objective={'MetricName': 'AUC'},
    max_candidates=250)

In [ ]:
automl_job.fit(inputs=dataset_path)